# TAREA:


# Regresion Lineal
## Basado en Christensen, L y W. Greene (1976), "Economies of Scale in U.S. Electric Power Generation", Journal of Political Economy, Vol. 84, No. 4, pp. 655-676

A continuación, resuleva cada una las instrucciones que se le señalan

In [36]:
# Importamos Dependencias o Bibliotecas necesarias:
import pyreadstat # librería para leer formato ".dta"
import pandas as pd # librería para manipulación de datos
import numpy as np # Librería para operaciones matemáticas
import matplotlib # Librería para graficar
from matplotlib import pyplot as plt # Librería para graficar
import statsmodels.api as sm # Librería para análisis estadístico
from IPython.display import Image # Librería para importar imagénes
from statsmodels.formula.api import ols # Librería para pruebas de hipotesis

#
import warnings
warnings.filterwarnings('ignore')

### La base de datos contiene información de 100 empresa de generación eléctrica, reportada en las siguientes variables:

* **firmid:** ID de la empresa
* **cost:** Total costs in MM USD
* **output:** output billion KwH
* **plabor:** price of labor
* **pfuel:** price of fuel
* **pkap:** price of capital
* **labshr:** participación del trabajo en el costo
* **kapshr:** participación del capital en el costo

## 1. Importe la base de DATOS:

In [69]:
import pyreadstat
dtafile = 'Data/cgreene76 (1).dta'


# La función read_dat devuelve dos objetos: el dataframe y los metadatos
df, meta = pyreadstat.read_dta("cgreene76 (1).dta")
df.head()
#El head fue para verificar si esta bien

,firmid,costs,output,plabor,pkap,pfuel,labshr,kapshr
0,1.0,0.2130,8.0,6869.470215,64.945000,18.000000,0.3291,0.4197
1,20.0,0.4887,14.0,5438.890137,86.110001,34.150002,0.2994,0.4571
2,17.0,0.6159,50.0,9204.240234,90.470001,32.070000,0.2963,0.0981
3,14.0,0.7606,65.0,8971.889648,41.243000,28.539000,0.2802,0.1282
4,28.0,0.6363,67.0,6696.500000,58.257999,25.400000,0.2447,0.3674


In [70]:
# Muestre los datos importados
df.tail(10)

,firmid,costs,output,plabor,pkap,pfuel,labshr,kapshr
89,127.0,111.867996,23217.0,6873.729980,83.879997,33.394402,0.0849,0.2007
90,142.0,125.335999,24001.0,8047.350098,74.372002,33.093201,0.0998,0.2457
91,137.0,183.231995,27118.0,9914.360352,78.480003,41.757801,0.1280,0.2265
92,130.0,87.101501,27708.0,6378.229980,63.599998,20.299999,0.1060,0.2257
93,144.0,240.514008,29613.0,9312.929688,81.750000,41.887199,0.1561,0.2017
94,143.0,191.563004,30958.0,9810.099609,69.541000,36.307598,0.1636,0.1524
95,141.0,168.378006,34212.0,5683.830078,80.385002,40.528599,0.0651,0.1361
96,138.0,169.235001,38343.0,9117.160156,65.991997,31.589701,0.0663,0.2192
97,175.0,269.773010,46870.0,9761.379883,69.541000,33.199902,0.1594,0.2194
98,172.0,240.485992,53918.0,6068.870117,78.379997,31.195400,0.0966,0.1846


# Considere una función de costos del tipo translog, escrita como:

$$ln(C) = \alpha_0 +  \alpha_Y ln(Y) + \frac{1}{2} \gamma_{YY} (ln(Y))^2 + \sum_k \alpha_k ln(P_{k}) + \frac{1}{2} \sum_{k} \sum_{j} \gamma_{kj} ln(P_k) ln(P_j) + \sum_k \gamma_{Yk} ln(Y) ln(P_k) + \varepsilon_i$$

Este ecuación es equivalente a la que Christensen y Greene denominan Modelo A. Donde:
$\gamma_{kj} = \gamma_{jk}$, $C$ es el costo total, $Y$ es el producto, $P_k$'s y $P_j$'s son los precios de los factores. $k,j = $ precio de combustible, precio del capital y precio del trabajo.

Noten que la doble suma $\sum_{k} \sum_{j} \gamma_{kj} ln(P_k) ln(P_j)$ implica que el caso $(ln(P_k))^2$ y $(ln(P_j))^2$ son posibles.

## 2. Realice las transformaciones necesarias en los datos para estimar la ecuación anterior:

In [75]:
#Creamos la variable:
dataframe = df
df['Lcosts'] = np.log(dataframe['costs'])
df['Loutput'] = np.log(dataframe['output'])
df['Lplabor'] = np.log(dataframe['plabor'])


dataframe['One'] = 1


In [76]:
# Muestre su data frame con las variables adicionales necesarias
df.head()

,firmid,costs,output,plabor,pkap,pfuel,labshr,kapshr,Lcosts,Loutput,Lplabor,One
0,1.0,0.2130,8.0,6869.470215,64.945000,18.000000,0.3291,0.4197,-1.546463,2.079442,8.834842,1
1,20.0,0.4887,14.0,5438.890137,86.110001,34.150002,0.2994,0.4571,-0.716006,2.639057,8.601330,1
2,17.0,0.6159,50.0,9204.240234,90.470001,32.070000,0.2963,0.0981,-0.484671,3.912023,9.127420,1
3,14.0,0.7606,65.0,8971.889648,41.243000,28.539000,0.2802,0.1282,-0.273648,4.174387,9.101852,1
4,28.0,0.6363,67.0,6696.500000,58.257999,25.400000,0.2447,0.3674,-0.452085,4.204693,8.809340,1


## 3. Realice la estimación de la regresión de la función translog:

In [103]:
# Definición de vector y matriz: Y y X
Y = ["Ltotcost"]
X = [["One","Loutput", "Lplabor", "Lpfuel", "Lpkap","Lcost" ,"Loutput" , "Lplabor" ]]
#
X

[['One',
  'Loutput',
  'Lplabor',
  'Lpfuel',
  'Lpkap',
  'Lcost',
  'Loutput',
  'Lplabor']]

In [104]:
# Resporte de la estimación:
modelo = sm.OLS(Y,X)
est2 = est.fit()
print(est2.summary())

UFuncTypeError: ufunc 'maximum' did not contain a loop with signature matching types (dtype('<U7'), dtype('<U7')) -> None

## 4. Pruebe cada una de las siguientes hipótesis

En cada caso escriba sus conclusiones respecto a la prueba y qué significado tiene para la estimación, de acuerdo con el trabajo de Christensen y Greene (1976)

In [90]:
# Establecemos la formula de la ecuación estimada:
formula = 'Ltotcost ~  One + Loutput + Lplabor + Lpfuel + Lpkap + Lcost + Loutput + Lplabor '
results = ols(formula, DataFrame).fit()


NameError: name 'DataFrame' is not defined

### 4.1
$$H_0: \sum_k \alpha_k = 1$$

In [ ]:
# Establecemos la hipótesis
hypotheses_1 = ' +  +  = 1'

In [ ]:
# Prueba de la hipótesis:
t_test_1 = results.t_test(hypotheses_1)
print(t_test_1)

### 4.2
$$H_0: \sum_k \gamma_{Yk} = 0$$

In [ ]:
# Establecemos la hipótesis
hypotheses_2 = ' +  +  = 1'

In [ ]:
# Prueba de la hipótesis:
t_test_2 = results.t_test(hypotheses_2)
print(t_test_2)

### 4.3
$$H_0: \sum_{k} \gamma_{kj} = \sum_{j} \gamma_{kj} = \sum_{k} \sum_{j} \gamma_{kj} = 0$$

In [ ]:
# Establecemos la hipótesis
hypotheses_3 = ' +  +  = 1'

In [ ]:
# Prueba de la hipótesis:
t_test_3 = results.t_test(hypotheses_3)
print(t_test_3)

## 5. Dadas sus conclusiones en las pruebas de hipótesis anteriores, ¿considere si debe ajustar la función de costos del tipo translog descrita anteriormente?, en su caso ajuste la siguiente expresión a la forma que debería estimar:

$$ln(C) = \alpha_0 +  \alpha_Y ln(Y) + \frac{1}{2} \gamma_{YY} (ln(Y))^2 + \sum_k \alpha_k ln(P_{k}) + \frac{1}{2} \sum_k \sum_j \gamma_{kj} ln(P_k) ln(P_j) + \sum_k \gamma_{Yk} ln(Y) ln(P_k) + \varepsilon_i$$

## Estime la nueva propuesta

In [ ]:
# Definición de vector y matriz: Y y X
Y = ["Ltotcost"]
X = [["One","Loutput", "Lplabor", "Lpfuel", "Lpkap", , , ,]]
#
X

In [ ]:
# Resporte de la estimación:
est = sm.OLS(Y,X)
est2 = est.fit()
print(est2.summary())

## 6. Determine el valor del Costo Total Estimado y el Costo Medio Estimado considerado cualquiera de las estimaciones que considere pertinente

In [ ]:
# Rcuperamos de la regresión el valor del Log del costo estimado
 = est2.predict(X)
# Anti-log:
Y = np.exp( )
# Colocamos en el Data Frame:
['totcost_e'] = Y
['avgcost_e'] = ["totcost_e"]/  ["output"]
.head()

## 7. Gráfique su estimación de Costo Medio y el Costo Medio observado

In [ ]:
# graficamos resultados:
plt.scatter(   .output, .avgcost, s = , color = "")
plt.scatter( .output, .avgcost_e, s = , color = "")
plt.title("Gráfico de dispersión PRoducto Vs Avg Cost y Avg Cost estimado")

#
plt.show()

# Guarde la imagen en un archivo PNG:
save()

## 8. Guarde el Data Frame en un archivo CSV: